In [13]:
import sys

if 'google.colab' in sys.modules:
    !git clone https://github.com/theoliver7/uzh-aml.git
    %cd uzh-aml
    !git checkout training-notebook
    %load_ext tensorboard
    !pip install -r colab-requirements.txt
    from torch.utils.tensorboard import SummaryWriter


In [14]:
import os

import torch
import torch.nn.functional as F
from torch.utils.data import random_split
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [15]:
args = {
    'device': "gpu",
    'seed': 42,
    'batch_size': 120,
}


In [19]:
torch.set_printoptions(edgeitems=10)

if torch.cuda.is_available():
    args["device"] = 'cuda'
# SET THE RANDOM SEED
torch.manual_seed(args['seed'])

dataset = TUDataset(os.path.join('../../data', "DD"), name="DD", use_node_attr=True)
args["num_classes"] = dataset.num_classes
args["num_features"] = dataset.num_features
num_test = int(len(dataset) * 0.1)
num_train = len(dataset) - num_test
_, test_dataset = torch.utils.data.random_split(dataset, [num_train, num_test])

{'device': 'cuda', 'seed': 42, 'batch_size': 120, 'num_classes': 2, 'num_features': 89}


In [17]:
def compute_test(model, loader):
    model.eval()
    correct = 0.0
    loss_test = 0.0
    for data in loader:
        data = data.to(args["device"])
        out = model(data)
        pred = out.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
        loss_test += F.nll_loss(out, data.y).item()
    return correct / len(loader.dataset), loss_test

In [18]:
test_loader = DataLoader(test_dataset, batch_size=args["batch_size"], shuffle=False)
model = torch.load("4.pth")
model.eval()
acc_test, loss_test = compute_test(model, test_loader)
# Output cross-validation and test results
print(f"Model accuracy: {acc_test}")
print(f"Model loss: {loss_test}")

Model accuracy: 0.6153846153846154
Model loss: 0.6706265211105347
